# Imports

In [ ]:
import os
import shutil
import numpy as np
import torch
from qYOLO.qyolo import QTinyYOLOv2, YOLOout, readAnchors

# Import Network

> pretrained = True<br>
img_dir = "./../../Dataset/images"<br>
lbl_dir = "./../../Dataset/labels"<br>
weight_bit_width = 8<br>
act_bit_width    = 8<br>
n_anchors        = 5<br>
n_epochs         = 10<br>
batch_size       = 10<br>

> net = QTinyYOLOv2(n_anchors, weight_bit_width, act_bit_width)<br>
if pretrained:<br>
    net_path = f"./train_out/trained_net_W{weight_bit_width}A{act_bit_width}_a{n_anchors}.pth"<br>
    net.load_state_dict(torch.load(net_path))<br>
    anchors_path = f"./anchors_W{weight_bit_width}A{act_bit_width}_a{n_anchors}.txt"<br>
    anchors = torch.zeros((n_anchors, 2))<br>
    with open(anchors_path) as f:<br>
        for n, data in enumerate(f.readlines()):<br>
            anchors[n] = torch.from_numpy(np.array(data.split(',')).astype(float))<br>
        f.close()<br>
else:<br>
    net, anchors = train(img_dir, lbl_dir, weight_bit_width=weight_bit_width, act_bit_width=act_bit_width, n_anchors=n_anchors, n_epochs=n_epochs, batch_size=batch_size)

In [ ]:
img_dir = "./../../Dataset/images"
lbl_dir = "./../../Dataset/labels"
weight_bit_width = 8
act_bit_width    = 8
n_anchors        = 5
n_epochs         = 10
batch_size       = 1

anchors = readAnchors(f'./../../train_out/anchors_W8A8_a5.txt')
print(anchors)
net = QTinyYOLOv2(n_anchors, weight_bit_width, act_bit_width)
net_path = f'./../../train_out/trained_net_W8A8_a5.pth'
net.load_state_dict(torch.load(net_path))

# FINN

## Imports

In [ ]:
# FINN-Brevitas imports
from brevitas.onnx import export_finn_onnx as exportONNX
# from brevitas.onnx import export_brevitas_onnx as exportONNX

from brevitas.export.onnx.generic.manager import BrevitasONNXManager

# ONNX libraries
import onnx
import onnx.numpy_helper as nph
import onnxruntime as rt

# Network display methods - Netron
from finn.util.visualization import showInNetron

# FINN Network Preperation imports
from finn.core.modelwrapper import ModelWrapper
from finn.core.datatype import DataType
from qonnx.util.cleanup import cleanup_model
from finn.util.pytorch import ToTensor
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from finn.transformation.merge_onnx_models import MergeONNXModels
from finn.transformation.streamline import Streamline
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants, MakeMaxPoolNHWC, MoveTransposePastScalarMul, MoveMulPastDWConv
from finn.transformation.streamline.absorb import AbsorbTransposeIntoMultiThreshold, AbsorbConsecutiveTransposes, AbsorbSignBiasIntoMultiThreshold, AbsorbMulIntoMultiThreshold
from finn.transformation.general import ConvertDivToMul, RemoveUnusedTensors
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.make_input_chanlast import MakeInputChannelsLast
from finn.transformation.fpgadataflow.convert_to_hls_layers import InferThresholdingLayer, InferConvInpGen, InferChannelwiseLinearLayer, InferStreamingMaxPool, InferQuantizedStreamingFCLayer
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition
from finn.custom_op.registry import getCustomOp

from finn.builder.build_dataflow import DataflowBuildConfig, build_dataflow_cfg
import finn.builder.build_dataflow_config as build_cfg
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild

## Brevitas Export

In [ ]:
onnx_dir = f'./onnx_W{weight_bit_width}A{act_bit_width}_a{n_anchors}/'
os.makedirs(onnx_dir, exist_ok=True)

exportONNX(net, (1, 3, 640, 640), onnx_dir + "og_net.onnx")
net_onnx_path = f'./../../train_out/trained_net_W8A8_a5.onnx'

model = ModelWrapper(onnx_dir + "og_net.onnx")
# model = ModelWrapper(net_onnx_path)
model = cleanup_model(model)
model = model.transform(ConvertQONNXtoFINN())

model.save(onnx_dir + "tidy_net.onnx")
showInNetron(onnx_dir + "tidy_net.onnx")

## Networks Preperation

### Add Pre/Post-Processing

In [ ]:
model = ModelWrapper(onnx_dir + "tidy_net.onnx")

# pre-processing
in_name = model.graph.input[0].name
in_shape = model.get_tensor_shape(in_name)
totensor = ToTensor()
exportONNX(totensor, in_shape, onnx_dir + "preproc_net.onnx")
pre_model = ModelWrapper(onnx_dir + "preproc_net.onnx")
model = model.transform(MergeONNXModels(pre_model))
in_name = model.graph.input[0].name
model.set_tensor_datatype(in_name, DataType["UINT8"])

# post-processing
# TODO - check if I can actually create the output layer
model = cleanup_model(model)
model = model.transform(ConvertQONNXtoFINN())

model.save(onnx_dir + "preproc_net.onnx")
showInNetron(onnx_dir + "preproc_net.onnx")

### Streamline

In [ ]:
model = ModelWrapper(onnx_dir + "preproc_net.onnx")

model = model.transform(MakeInputChannelsLast())
model = model.transform(MoveScalarLinearPastInvariants())
model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
model = model.transform(MakeMaxPoolNHWC()) 
model = model.transform(AbsorbTransposeIntoMultiThreshold())
model = model.transform(AbsorbConsecutiveTransposes())

model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())
model = cleanup_model(model)

model.save(onnx_dir + "streamline_net.onnx")
showInNetron(onnx_dir + "streamline_net.onnx")

### Convert to HLS

In [ ]:
model = ModelWrapper(onnx_dir + "streamline_net.onnx")

model = model.transform(InferConvInpGen())
model = model.transform(InferQuantizedStreamingFCLayer())
model = model.transform(InferStreamingMaxPool())
model = model.transform(InferThresholdingLayer())
model = cleanup_model(model)

model.save(onnx_dir + "hls_net.onnx")
showInNetron(onnx_dir + "hls_net.onnx")

### Create Dataflow Partition
Failed to remove the final mul and transpose layer, so I just remove the during the partition

In [ ]:
model = ModelWrapper(onnx_dir + "hls_net.onnx")

parent_model = model.transform(CreateDataflowPartition())

parent_model.save(onnx_dir + "parent_net.onnx")
showInNetron(onnx_dir + "parent_net.onnx")

In [ ]:
parent_model = ModelWrapper(onnx_dir + "parent_net.onnx")

sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
model_filename = sdp_node.get_nodeattr("model")
model = ModelWrapper(model_filename)
model.rename_tensor(model.get_all_tensor_names()[-1], "global_out")
model = cleanup_model(model)

model.save(onnx_dir + "dataflow_net.onnx")
showInNetron(onnx_dir + "dataflow_net.onnx")

### Folding

In [ ]:
model = ModelWrapper(onnx_dir + "dataflow_net.onnx")
layers = model.get_finn_nodes()
names = model.get_all_tensor_names()

for i, layer in enumerate(layers):
    temp_op = getCustomOp(layer)
    print(f"CustomOp wrapper of {layer.name}:")
    for item in temp_op.get_nodeattr_types():
        print(f"{item}: {temp_op.get_nodeattr_types()[item]} = {temp_op.get_nodeattr(item)}")
    print()

In [ ]:
model = ModelWrapper(onnx_dir + "dataflow_net.onnx")


model.save(onnx_dir + "folded_net.onnx")
showInNetron(onnx_dir + "folded_net.onnx")

# Hardware Build and Deployment

## Build Configurations

In [ ]:
out_dir = onnx_dir + "hw_out"
#Delete previous run results if exist
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
    print("Previous run results deleted!")

cfg_estimates = DataflowBuildConfig(auto_fifo_depths= True, board="ZCU102", default_mem_mode='constant',
                                    output_dir=out_dir, synth_clk_period_ns= 10.0, target_fps=50,
                                    steps=build_cfg.estimate_only_dataflow_steps,
                                    generate_outputs=[build_cfg.DataflowOutputType.ESTIMATE_REPORTS])

# cfg = DataflowBuildConfig(auto_fifo_depths= True, board="ZCU102", default_mem_mode= 'decoupled',
#                           output_dir=out_dir, synth_clk_period_ns= 10.0, target_fps=50,
#                           steps=build_cfg.estimate_only_dataflow_steps,
#                           build_cfg.DataflowOutputType.ESTIMATE_REPORTS)

## Build

In [ ]:
model = ModelWrapper(onnx_dir + "folded_net.onnx")

build_dataflow_cfg(onnx_dir + "folded_net.onnx", cfg_estimates)

# model = model.transform(ZynqBuild(platform = "ZCU102", period_ns = 10))
# model.save(onnx_dir + "hw_net.onnx")